In [171]:
import pandas as pd
import numpy as np 
import matplotlib.pylab as plt
import os

from gensim import corpora, models, similarities, utils

%autoreload 
from updating_biz_df import *

DATA = '../..'
TMP = '../../tmp'
#!ls ../../

In [ ]:
rf = pd.read_pickle(DATA+'/reviews.pkl')

In [87]:
bf = pd.read_csv(DATA+'/csvs/biz_details.csv')

In [118]:
st = rf.groupby( [ "biz_id", "stars"] )
sf = st.stars.size().unstack("stars").fillna(0)
sf.reset_index(level=sf.index.names, inplace=True)

sf.columns = ['biz_id', '1stars', '2stars', '3stars', '4stars', '5stars']

In [119]:
sf['star_cnts'] = list(sf[[1,2,3,4,5]].values)
sf.head()

,biz_id,1stars,2stars,3stars,4stars,5stars,star_cnts
0,100-de-agave-mexican-grill-and-cantina-denver,31.0,24.0,34.0,85.0,129.0,"[31.0, 24.0, 34.0, 85.0, 129.0]"
1,12-at-madison-denver,0.0,1.0,2.0,1.0,20.0,"[0.0, 1.0, 2.0, 1.0, 20.0]"
2,20th-street-caf%C3%A9-denver-2,3.0,9.0,13.0,51.0,59.0,"[3.0, 9.0, 13.0, 51.0, 59.0]"
3,3-girasoles-denver-2,0.0,1.0,2.0,4.0,11.0,"[0.0, 1.0, 2.0, 4.0, 11.0]"
4,3-guys-pies-denver,26.0,19.0,13.0,44.0,64.0,"[26.0, 19.0, 13.0, 44.0, 64.0]"


In [120]:
tf = pd.merge(sf, bf, on='biz_id')

In [154]:
#take advanatage of column that needs to be dropped, but we'll be adding hdp probs iterativel and will need a column to full
tf=tf.rename(columns = {'Unnamed: 0':'hdp_probs'})
tf['hdp_probs'] = np.nan
tf.head()

,biz_id,1stars,2stars,3stars,4stars,5stars,star_cnts,hdp_probs,addressCountry,addressLocality,...,priceRange,ratingValue,ratings_trend,reviewCount,servesCuisine,streetAddress,telephone,type,web_title,yelp_since
0,100-de-agave-mexican-grill-and-cantina-denver,31.0,24.0,34.0,85.0,129.0,"[31.0, 24.0, 34.0, 85.0, 129.0]",NaN,US,Denver,...,$11-30,4.0,"{""2016"": [[0, 3.5], [1, 3.0], [2, 3.5], [3, 4....",293,Mexican,975 Lincoln St,1.303731e+10,Restaurant,100% De Agave Mexican Grill and Cantina - Orde...,2013
1,12-at-madison-denver,0.0,1.0,2.0,1.0,20.0,"[0.0, 1.0, 2.0, 1.0, 20.0]",NaN,US,Denver,...,NaN,4.5,"{""2017"": [[2, 4.5], [3, 4.5], [4, 4.5], [5, 4....",24,American (Traditional),1160 Madison St,1.720216e+10,Restaurant,"12 at Madison - Last Updated June 7, 2017 - 31...",2016
2,20th-street-caf%C3%A9-denver-2,3.0,9.0,13.0,51.0,59.0,"[3.0, 9.0, 13.0, 51.0, 59.0]",NaN,US,Denver,...,Under $10,4.0,"{""2016"": [[0, 4.5], [1, 4.5], [2, 4.5], [3, 4....",133,Diners,1123 20th St,1.303296e+10,Restaurant,20th Street Café - 96 Photos & 133 Reviews - D...,2008
3,3-girasoles-denver-2,0.0,1.0,2.0,4.0,11.0,"[0.0, 1.0, 2.0, 4.0, 11.0]",NaN,US,Denver,...,Under $10,4.5,"{""2016"": [[5, 4.5], [6, 4.5], [7, 4.5], [8, 4....",17,Mexican,2940 E Bruce Randolph Ave,1.720690e+10,Restaurant,3 Girasoles - 17 Reviews - Mexican - 2940 E Br...,2014
4,3-guys-pies-denver,26.0,19.0,13.0,44.0,64.0,"[26.0, 19.0, 13.0, 44.0, 64.0]",NaN,US,Denver,...,Under $10,3.5,"{""2016"": [[0, 4.0], [1, 3.5], [2, 3.5], [3, 3....",159,Pizza,1501 E Colfax Ave,1.303623e+10,Restaurant,3 Guys Pies - Order Food Online - 63 Photos & ...,2010


In [160]:
#test    
biz_id = 'not-there'
topic_prob_extractor(load_HDP_model(biz_id, path=MODEL_PATH),df=False)
biz_id = '7-leguas-mexican-grille-denver-4'
topic_prob_extractor(load_HDP_model(biz_id, path=MODEL_PATH),df=False)

../../preprocessed/not-there_hdp.model Not found!


[0.13346584804682202,
 0.12734280319070418,
 0.12625297988408088,
 0.12067896047403234,
 0.14705017086874572,
 0.14013675898035763,
 0.1211390055835703,
 0.13029426418132803,
 0.13495988062043068,
 0.1354358620990877,
 0.12848483511341963,
 0.12461267543826267,
 0.12678263896120048,
 0.140411065526691,
 0.13359655468447557,
 0.12773195151212022,
 0.13328430079804138,
 0.12367167189779101,
 0.13525400338063082,
 0.13207005363299382,
 0.14473674559088651,
 0.12423640499596478,
 0.1207682696042184,
 0.14371129856635589,
 0.13961121240320554,
 0.11903853033612383,
 0.12280845178092605,
 0.14761040068466924,
 0.1327107810621726,
 0.13949574547432103]

In [165]:
def f(df): 
    biz_id = str(df['biz_id'])
    p = topic_prob_extractor(load_HDP_model(biz_id, path=MODEL_PATH),df=False)
    if p :
        return p
    else :
        return np.nan
    
# we didn't run models on those with reviews less than X 
# so there are not founds
tf['hdp_probs'] = tf.apply(lambda x: f(x), axis=1)



../../preprocessed/12-at-madison-denver_hdp.model Not found!
../../preprocessed/3-girasoles-denver-2_hdp.model Not found!
../../preprocessed/3-sons-bbq-denver_hdp.model Not found!
../../preprocessed/5-buck-drinkery-denver_hdp.model Not found!
../../preprocessed/9thai-restaurant-denver_hdp.model Not found!
../../preprocessed/a-brothers-deli-denver_hdp.model Not found!
../../preprocessed/a-taste-of-haiti-denver_hdp.model Not found!
../../preprocessed/a-taste-of-the-philippines-denver-2_hdp.model Not found!
../../preprocessed/acero-denver_hdp.model Not found!
../../preprocessed/ajuua-denver_hdp.model Not found!
../../preprocessed/ali-baba-grill-denver-3_hdp.model Not found!
../../preprocessed/alto-restaurant-denver_hdp.model Not found!
../../preprocessed/andres-confiserie-suisse-denver_hdp.model Not found!
../../preprocessed/antojitos-colombiano-denver-2_hdp.model Not found!
../../preprocessed/arch-pizza-denver-3_hdp.model Not found!
../../preprocessed/areyto-puertorrican-food-denver_hdp.

../../preprocessed/la-potranca-taqueria-denver_hdp.model Not found!
../../preprocessed/la-scala-trattoria-denver_hdp.model Not found!
../../preprocessed/lancers-diner-denver_hdp.model Not found!
../../preprocessed/larimer-beer-hall-denver_hdp.model Not found!
../../preprocessed/las-fajitas-mexican-restaurant-denver_hdp.model Not found!
../../preprocessed/las-tortugas-denver-3_hdp.model Not found!
../../preprocessed/lazo-empanadas-denver-4_hdp.model Not found!
../../preprocessed/le-peep-denver_hdp.model Not found!
../../preprocessed/lechugas-italian-restaurant-denver-2_hdp.model Not found!
../../preprocessed/little-carmines-denver-2_hdp.model Not found!
../../preprocessed/little-europe-denver-2_hdp.model Not found!
../../preprocessed/little-spoons-denver_hdp.model Not found!
../../preprocessed/los-gallitos-taqueria-denver_hdp.model Not found!
../../preprocessed/los-molcajetes-taqueria-denver_hdp.model Not found!
../../preprocessed/los-molinos-restaurant-denver_hdp.model Not found!
../..

../../preprocessed/tambien-denver_hdp.model Not found!
../../preprocessed/taqueria-mi-pueblo-denver_hdp.model Not found!
../../preprocessed/teriyaki-madness-denver_hdp.model Not found!
../../preprocessed/territory-kitchen-bar-denver_hdp.model Not found!
../../preprocessed/thai-curry-cuisine-denver_hdp.model Not found!
../../preprocessed/thai-spice-cafe-denver_hdp.model Not found!
../../preprocessed/the-bank-bar-and-grill-denver_hdp.model Not found!
../../preprocessed/the-colorado-pig-rig-denver-2_hdp.model Not found!
../../preprocessed/the-continental-room-and-lounge-denver_hdp.model Not found!
../../preprocessed/the-crumb-cafe-denver_hdp.model Not found!
../../preprocessed/the-dancing-crab-denver_hdp.model Not found!
../../preprocessed/the-griffin-tavern-denver_hdp.model Not found!
../../preprocessed/the-hound-denver-3_hdp.model Not found!
../../preprocessed/the-house-korean-bbq-denver_hdp.model Not found!
../../preprocessed/the-long-i-pie-shop-denver_hdp.model Not found!
../../prepro

In [166]:
tf[['biz_id','star_cnts','hdp_probs','ratingValue']].head()



,biz_id,star_cnts,hdp_probs,ratingValue
0,100-de-agave-mexican-grill-and-cantina-denver,"[31.0, 24.0, 34.0, 85.0, 129.0]","[0.0465603239697, 0.0442118345098, 0.041815835...",4.0
1,12-at-madison-denver,"[0.0, 1.0, 2.0, 1.0, 20.0]",NaN,4.5
2,20th-street-caf%C3%A9-denver-2,"[3.0, 9.0, 13.0, 51.0, 59.0]","[0.0672861106344, 0.0680105883199, 0.065648209...",4.0
3,3-girasoles-denver-2,"[0.0, 1.0, 2.0, 4.0, 11.0]",NaN,4.5
4,3-guys-pies-denver,"[26.0, 19.0, 13.0, 44.0, 64.0]","[0.0576562312717, 0.06243637116, 0.06696157629...",3.5


In [170]:
tf['hdp_probs'].dropna()[:10]

0     [0.0465603239697, 0.0442118345098, 0.041815835...
2     [0.0672861106344, 0.0680105883199, 0.065648209...
4     [0.0576562312717, 0.06243637116, 0.06696157629...
7     [0.0385055135607, 0.0362471626885, 0.030629124...
8     [0.0628963861249, 0.0669738411756, 0.066189770...
9     [0.133465848047, 0.127342803191, 0.12625297988...
10    [0.0654999010536, 0.0751366967271, 0.072450256...
11    [0.0727943584514, 0.0811721844721, 0.081556634...
16    [0.0544124240173, 0.0525839623252, 0.056755832...
17    [0.0438507308655, 0.0286388586773, 0.028046151...
Name: hdp_probs, dtype: object

In [172]:
def g(df): 
    biz_id = str(df['biz_id'])
    p = load_HDP_model(biz_id, path=MODEL_PATH)
    if p :
        return p.show_topics()
    else :
        return np.nan
    
# we didn't run models on those with reviews less than X 
# so there are not founds
tf['hdp_topics'] = tf.apply(lambda x: g(x), axis=1)

In [173]:
tf[['biz_id','star_cnts','hdp_probs','hdp_topics', 'ratingValue']].head()


,biz_id,star_cnts,hdp_probs,hdp_topics,ratingValue
0,100-de-agave-mexican-grill-and-cantina-denver,"[31.0, 24.0, 34.0, 85.0, 129.0]","[0.0465603239697, 0.0442118345098, 0.041815835...","[(0, 0.003*occasion + 0.003*yeah + 0.003*bleh ...",4.0
1,12-at-madison-denver,"[0.0, 1.0, 2.0, 1.0, 20.0]",NaN,NaN,4.5
2,20th-street-caf%C3%A9-denver-2,"[3.0, 9.0, 13.0, 51.0, 59.0]","[0.0672861106344, 0.0680105883199, 0.065648209...","[(0, 0.005*surprisingly + 0.004*blow + 0.004*m...",4.0
3,3-girasoles-denver-2,"[0.0, 1.0, 2.0, 4.0, 11.0]",NaN,NaN,4.5
4,3-guys-pies-denver,"[26.0, 19.0, 13.0, 44.0, 64.0]","[0.0576562312717, 0.06243637116, 0.06696157629...","[(0, 0.004*ball + 0.004*sooooo + 0.003*reviewe...",3.5


In [174]:
tf.to_pickle('biz-models.pkl')

In [4]:
model = models.LdaModel.load('preprocessed/7-leguas-mexican-grille-denver-4_lda.model')

In [40]:
model.show_topics()

[(0,
  '0.001*"food" + 0.001*"good" + 0.001*"great" + 0.001*"place" + 0.001*"mexican" + 0.001*"service" + 0.001*"staff" + 0.001*"friendly" + 0.001*"margarita" + 0.001*"time"'),
 (17,
  '0.002*"incredible" + 0.002*"colfax" + 0.002*"come" + 0.002*"walk" + 0.002*"house" + 0.002*"serve" + 0.002*"people" + 0.002*"hesitant" + 0.002*"pastor" + 0.002*"willow"'),
 (8,
  '0.002*"place" + 0.002*"food" + 0.002*"sweetness" + 0.002*"star" + 0.002*"order" + 0.002*"better" + 0.002*"meat" + 0.002*"temp" + 0.002*"competence" + 0.001*"lingual"'),
 (6,
  '0.002*"couldn" + 0.002*"time" + 0.002*"place" + 0.002*"little" + 0.002*"bean" + 0.002*"good" + 0.002*"chile" + 0.002*"enchilada" + 0.002*"service" + 0.002*"pleased"'),
 (2,
  '0.002*"good" + 0.002*"bean" + 0.002*"table" + 0.002*"nice" + 0.002*"salsa" + 0.002*"drink" + 0.002*"taco" + 0.002*"salad" + 0.002*"combo" + 0.002*"seafood"'),
 (16,
  '0.036*"food" + 0.036*"great" + 0.027*"service" + 0.027*"place" + 0.027*"good" + 0.018*"amazing" + 0.018*"wonderful

In [8]:
hdp = gensim.models.hdpmodel.HdpModel.load('preprocessed/7-leguas-mexican-grille-denver-4_hdp.model')

In [11]:
hdp2 = gensim.models.hdpmodel.HdpModel.load('preprocessed/sushi-den-denver_hdp.model')

In [13]:
hdp3 = gensim.models.hdpmodel.HdpModel.load('preprocessed/biker-jims-gourmet-dogs-denver-5_hdp.model')

In [9]:
hdp.show_topics()

[(0,
  '0.012*hate + 0.010*hidden + 0.008*dive + 0.007*far + 0.007*chile + 0.007*unlimited + 0.007*fresh + 0.006*picture + 0.006*instead + 0.006*lot + 0.006*suffered + 0.006*shelf + 0.006*black + 0.006*hesitant + 0.006*darn + 0.006*la + 0.006*good + 0.005*soupy + 0.005*bunch + 0.005*extremely'),
 (1,
  '0.011*fish + 0.008*large + 0.008*pretty + 0.008*especially + 0.007*star + 0.007*started + 0.007*eek + 0.006*wasn + 0.006*wall + 0.006*poblano + 0.006*real + 0.006*center + 0.006*stocked + 0.006*queso + 0.005*gratis + 0.005*bennie + 0.005*test + 0.005*wait + 0.005*dive + 0.005*open'),
 (2,
  '0.009*competence + 0.008*st + 0.008*personal + 0.008*flyover + 0.007*place + 0.007*organized + 0.007*stuffed + 0.006*burrito + 0.006*artsy + 0.006*cooking + 0.006*willing + 0.006*absolutely + 0.005*bad + 0.005*amazingly + 0.005*leguas + 0.005*traditional + 0.005*chef + 0.005*priced + 0.005*warmed + 0.005*beautiful'),
 (3,
  '0.009*recommendation + 0.007*park + 0.007*yummy + 0.007*employee + 0.007*pe